In [1]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

/Users/ron/Documents/github/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='llama3.2',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [9]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [10]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [11]:
llm('write that this is a test')

'This is a test.'

In [12]:
query = 'the course has already started, can I still enroll?'

In [13]:
response = rag(query)

In [14]:
response

'Based on the CONTEXT provided, I am unable to determine if you can enroll in the course after it has started. The most relevant section for this inquiry is:\n\n"Course - Can I still join the course after the start date?"\n\nThe answer provided was: "Yes, even if you don\'t register, you\'re still eligible to submit the homeworks."\n\nHowever, no further guidance on whether or not your enrollment may have a cutoff, or what actions could be taken to enroll at that time'

In [15]:
print(response)

Based on the CONTEXT provided, I am unable to determine if you can enroll in the course after it has started. The most relevant section for this inquiry is:

"Course - Can I still join the course after the start date?"

The answer provided was: "Yes, even if you don't register, you're still eligible to submit the homeworks."

However, no further guidance on whether or not your enrollment may have a cutoff, or what actions could be taken to enroll at that time


# docker command to pull llama3.2
# docker exec -it ollama bash
# ollama pull llama3.2